In [1]:
import pandas as pd
import numpy as np
import datetime as dt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)
pd.set_option('float_format', '{:f}'.format)


In [2]:
# READING DATA
# Diagnose data 
# DIAG = pd.read_csv(r"C:\Users\elena\Downloads\OneDrive_1_10-4-2020\APLD Raw Data\DIAG.txt.gz",sep="|")

#Procedure data
PX = pd.read_csv(r"C:\Users\elena\Downloads\OneDrive_1_10-4-2020\APLD Raw Data\PX.txt.gz",sep="|")

# Presecription data
RX = pd.read_csv(r"C:\Users\elena\Downloads\OneDrive_1_10-4-2020\APLD Raw Data\RX.txt.gz",sep="|")


# Diagnosis Code reference
BC_SN = pd.read_excel(r"C:\Users\elena\Downloads\OneDrive_1_10-4-2020\BC_SN ICD Code.xlsx")


# Reference for diagnosis, procedure and product
# DIAG_refer = pd.read_csv(r"C:\Users\elena\Downloads\OneDrive_1_10-4-2020\APLD Raw Data\DIAGNOSIS_reference_20200925.txt",sep='|')
Procedure_refer = pd.read_csv(r"C:\Users\elena\Downloads\OneDrive_1_10-4-2020\APLD Raw Data\PROCEDURE_reference_20200925.txt",sep='|')

# Drug lable 
drug_label = pd.read_csv(r"C:\Users\elena\Downloads\OneDrive_1_10-4-2020\Drug_table_with_type.csv",sep='|')
drug_label = drug_label.loc[:,"NDC":]

C:\Users\elena\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Table Joinings
PX_merged = PX.merge(Procedure_refer,how="left",on="PROCEDURE_CODE")
RX_merged = RX.merge(drug_label,how="left",on="NDC")

# Merge DIAGNOSIS_CODE with reference
# DIAG_merged = DIAG.merge(BC_SN, how = 'left', left_on='DIAGNOSIS_CODE', right_on='diagnosis_cd').drop(columns=["diagnosis_cd"])

In [4]:
# Dealing with the service date
def lookup(s):
    """ 
    This is a faster approach to convert string to datetime object
    """
    dates = {date:pd.to_datetime(date) for date in s.unique()}
    return s.apply(lambda v: dates[v])

PX_merged['SERVICE_DATE'] = lookup(PX_merged['SERVICE_DATE'])
RX_merged['SERVICE_DATE'] = lookup(RX_merged['SERVICE_DATE'])
# DIAG_merged['SERVICE_DATE'] = lookup(DIAG_merged['SERVICE_DATE'])

In [7]:
PX_merged = PX_merged[['PATIENT_ID','SERVICE_DATE','DIAGNOSIS_CODE','PROCEDURE_CODE','SVC_CRGD_AMT','UNIT_OF_SVC_AMT','PLACE_OF_SERVICE','PAY_TYPE','PROCEDURE_DESCRIPTION','PRC_DOSAGE_AMT']]

In [8]:
PX_merged = PX_merged.merge(BC_SN,how="left", left_on='DIAGNOSIS_CODE',right_on ='diagnosis_cd').drop(columns=['diagnosis_cd','DIAGNOSIS_CODE','version','sub_indication_cd'])

In [9]:
# Convert the service date to datetime object
def get_BC_SN_diff(df):
    df = df.sort_values(['PATIENT_ID','SERVICE_DATE'],ascending = (True, True))
    SN = df[df['indication_cd']=='SN']
    BC = df[df['indication_cd']=='BC']

    SN = SN.sort_values(by='SERVICE_DATE').drop_duplicates(subset=['PATIENT_ID'], keep='first')
    BC = BC.sort_values(by='SERVICE_DATE').drop_duplicates(subset=['PATIENT_ID'], keep='first')

    SN_temp = SN.rename(columns={'SERVICE_DATE':'SN_DIAG_DATE'})[['PATIENT_ID','SN_DIAG_DATE']]
    BC_temp = BC.rename(columns={'SERVICE_DATE':'BC_DIAG_DATE'})[['PATIENT_ID','BC_DIAG_DATE']]

    # Obtain the mBC classification
    temp = SN_temp.merge(BC_temp,how='outer',on='PATIENT_ID')
    temp['Diff'] = temp['SN_DIAG_DATE'] - temp['BC_DIAG_DATE']
    temp['Diff'] = temp['Diff'].dt.days
    df = df.merge(temp,how='left',on='PATIENT_ID')
    return df

In [10]:
PX_merged = get_BC_SN_diff(PX_merged)

In [12]:
# Patient_Date_Class: patient_ID, SERVICE_DATE, class
def Get_Patient_Date_Class(df):
    Patient_Date_Class = df[['PATIENT_ID','SERVICE_DATE','SN_DIAG_DATE','BC_DIAG_DATE','Diff']]
    Patient_Date_Class = Patient_Date_Class.drop_duplicates()
    Patient_Date_Class['SERVICE_SN_Diff'] = Patient_Date_Class['SERVICE_DATE']-Patient_Date_Class['SN_DIAG_DATE']
    Patient_Date_Class['SERVICE_SN_Diff'] = Patient_Date_Class['SERVICE_SN_Diff'].dt.days

    # No BC/SN record: -1
    # only have BC record / record before SN_DIAG_DATE: 0
    # only have SN record  / -30 <= Diff <=0 / record after SN_DIAG_DATE: 1
    # Diff<-30: -2
    conditions = [((Patient_Date_Class['SN_DIAG_DATE'].isna()) & (Patient_Date_Class['BC_DIAG_DATE'].isna())),
                 ((Patient_Date_Class['SN_DIAG_DATE'].isna()) & (Patient_Date_Class['BC_DIAG_DATE'].notnull())),
                 ((Patient_Date_Class['SN_DIAG_DATE'].notnull()) & (Patient_Date_Class['BC_DIAG_DATE'].isna())),
                 ((Patient_Date_Class['Diff']>0) & (Patient_Date_Class['SERVICE_SN_Diff']<0)),
                  ((Patient_Date_Class['Diff']>0) & (Patient_Date_Class['SERVICE_SN_Diff']>=0)),
                 ((Patient_Date_Class['Diff']<=0) & (Patient_Date_Class['Diff']>=-30)),
                  (Patient_Date_Class['Diff']<-30)
                 ]
    choices =[-1,0,1,0,1,1,-2]
    Patient_Date_Class['class'] = np.select(conditions,choices,default=5)
    return Patient_Date_Class[['PATIENT_ID','SERVICE_DATE','class']]

PX_patient_date_class = Get_Patient_Date_Class(PX_merged)

In [13]:
PX_patient_date_class.head()

,PATIENT_ID,SERVICE_DATE,class
0,29388541,2017-02-06,0
2,29388541,2017-02-24,0
3,29388541,2017-03-07,0
5,29388541,2017-03-20,0
6,29388541,2017-03-24,0


In [14]:
PX_merged = PX_merged.merge(PX_patient_date_class,how="left",on=['PATIENT_ID','SERVICE_DATE'])
PX_merged = PX_merged.drop(columns=['indication_cd','SN_DIAG_DATE','BC_DIAG_DATE','Diff'])

In [15]:
def time_feature(df):
    min_dates = df.groupby(['PATIENT_ID'])['SERVICE_DATE'].min()
    df['Earliest_DIAG_Date'] = df.apply(lambda row: min_dates.loc[row['PATIENT_ID']], axis=1)

    df['Days_Diff'] = df['SERVICE_DATE'] - df['Earliest_DIAG_Date']
    df['Week_Diff'] = df['Days_Diff']/np.timedelta64(1,"W")
    df['Month_Diff'] = df['Days_Diff']/np.timedelta64(1,"M")
    df['Year_Diff'] = df['Days_Diff']/np.timedelta64(1,'Y')
    df['Days_Diff'] = df['Days_Diff'].dt.days

    return df

In [16]:
PX_merged = time_feature(PX_merged)

In [17]:
PX_merged['CHEMOTHERAPY'] = np.where(PX_merged['PROCEDURE_DESCRIPTION'].str.contains("CHEMOTHERAPY"),1,0)
PX_merged['CYCLOPHOSPHAMIDE'] = np.where(PX_merged['PROCEDURE_DESCRIPTION'].str.contains("CYCLOPHOSPHAMIDE"),1,0)
PX_merged['LYMPH'] = np.where(PX_merged['PROCEDURE_DESCRIPTION'].str.contains("LYMPH"),1,0)
PX_merged['GENE'] = np.where(PX_merged['PROCEDURE_DESCRIPTION'].str.contains("GENE"),1,0)
PX_merged['IMMUNOHISTOCHEMISTRY'] = np.where(PX_merged['PROCEDURE_DESCRIPTION'].str.contains("IMMUNOHISTOCHEMISTRY"),1,0)
PX_merged['IMMUNOCYTOCHEMISTRY'] = np.where(PX_merged['PROCEDURE_DESCRIPTION'].str.contains("IMMUNOCYTOCHEMISTRY"),1,0)
PX_merged['IMMUNOASSAY'] = np.where(PX_merged['PROCEDURE_DESCRIPTION'].str.contains("IMMUNOASsortSAY"),1,0)
PX_merged['METASTATIC'] = np.where(PX_merged['PROCEDURE_DESCRIPTION'].str.contains("METASTATIC"),1,0)
PX_merged['SURGI'] = np.where(PX_merged['PROCEDURE_DESCRIPTION'].str.contains("SURGI"),1,0)
PX_merged['MASTECTOMY'] = np.where(PX_merged['PROCEDURE_DESCRIPTION'].str.contains("MASTECTOMY"),1,0)
PX_merged['SUCCINYLCHOLINE CHLORIDE'] = np.where(PX_merged['PROCEDURE_DESCRIPTION'].str.contains("SUCCINYLCHOLINE CHLORIDE"),1,0)

In [18]:
PX_merged['PRC_DOSAGE_AMT'] = PX_merged.PRC_DOSAGE_AMT.fillna(0)
AMT =  pd.DataFrame(PX_merged.groupby(['PATIENT_ID','SERVICE_DATE','PROCEDURE_CODE'])[['SVC_CRGD_AMT','UNIT_OF_SVC_AMT',"PRC_DOSAGE_AMT"]].sum())
AMT.reset_index(inplace=True)

In [19]:
PX_merged = PX_merged.drop(columns=['PROCEDURE_DESCRIPTION','SVC_CRGD_AMT','UNIT_OF_SVC_AMT','PRC_DOSAGE_AMT'])
PX_merged = PX_merged.merge(AMT,how='left',on=['PATIENT_ID','SERVICE_DATE','PROCEDURE_CODE'])

In [28]:
PAY_TYPE_dummies.head()

,index,PAY_TYPE_MEDICAID,PAY_TYPE_NONE,PAY_TYPE_THIRD PARTY
0,0,0,0,1
1,1,0,0,1
2,2,0,0,1
3,3,0,0,1
4,4,0,0,1


In [22]:
PX_merged['PLACE_OF_SERVICE'] = PX_merged['PLACE_OF_SERVICE'].replace({np.nan: 'NONE'})
PX_merged['PAY_TYPE'] = PX_merged['PAY_TYPE'].replace({np.nan:'NONE'})

Place_dummies = pd.get_dummies(PX_merged.PLACE_OF_SERVICE, prefix='Place')
PAY_TYPE_dummies = pd.get_dummies(PX_merged.PAY_TYPE,prefix='PAY_TYPE')

Place_dummies.reset_index(inplace=True)
PAY_TYPE_dummies.reset_index(inplace=True)

PX_merged.reset_index(inplace=True)

In [30]:
PX_merged = PX_merged.merge(Place_dummies,how='left',on="index").merge(PAY_TYPE_dummies,how="left",on="index")

In [50]:
# Get number of procedure per patient per day
PX_count = pd.DataFrame(PX_merged.groupby(['PATIENT_ID','SERVICE_DATE']).size())
PX_count.reset_index(inplace=True)
PX_count = PX_count.rename(columns={0:"PX_count"})
PX_count = PX_count.drop(columns=['index'])

In [97]:
PX_count.head()

,PATIENT_ID,SERVICE_DATE,PX_count
0,29388541,2017-02-06,2
1,29388541,2017-02-24,1
2,29388541,2017-03-07,2
3,29388541,2017-03-20,1
4,29388541,2017-03-24,1


In [85]:
PX_merged['PX_in-between'] = np.where(PX_merged.loc[:,'CHEMOTHERAPY':'SUCCINYLCHOLINE CHLORIDE'].sum(axis=1)==0,1,0)
PX_merged['PX_Severe'] = np.where(PX_merged.loc[:,'CHEMOTHERAPY':'METASTATIC'].sum(axis=1)!=0,1,0)
PX_merged['PX_light'] = np.where(((PX_merged['PX_in-between']==0) & (PX_merged['PX_Severe']==0)),1,0)

In [94]:
PX_temp1 = PX_merged[['PATIENT_ID','SERVICE_DATE','class','Earliest_DIAG_Date','Days_Diff','Week_Diff','Year_Diff']]

In [91]:
PX_temp = PX_merged[list(PX_merged.loc[:,'PATIENT_ID':'SERVICE_DATE']) + list(PX_merged.loc[:,'CHEMOTHERAPY':'PX_light'])]

In [93]:
PX_temp = pd.DataFrame(PX_temp.groupby(['PATIENT_ID','SERVICE_DATE']).sum())

In [95]:
PX_final = PX_temp1.merge(PX_temp,how="left",on=['PATIENT_ID','SERVICE_DATE']).merge(PX_count,how="left",on=['PATIENT_ID','SERVICE_DATE'])

In [101]:
PX_final = PX_final.drop_duplicates(subset=['PATIENT_ID','SERVICE_DATE'])

In [168]:
PX_final.shape

(1451201, 64)

### Drug info

In [106]:
RX_column = ['PATIENT_ID','SERVICE_DATE','NDC','type','REFILL_CODE','DRUG_STRENGTH','DSPNSD_QTY','DAYS_SUPPLY']
RX_temp = RX_merged[RX_merged['type']!='OTHER'][RX_column]
RX_temp = RX_temp.drop_duplicates()

In [131]:
# Get the drug type dummy variables
drug_type_dummies =pd.get_dummies(RX_temp.type,prefix='drug_type')
drug_type_dummies.reset_index(inplace=True)
RX_temp.reset_index(inplace=True)
RX_temp = RX_temp.merge(drug_type_dummies,how="left",on="index")

In [174]:
New_RX_count.head()

,PATIENT_ID,SERVICE_DATE,New_RX
0,29388541,2017-08-15,1
1,29388541,2017-11-06,0
2,29388541,2018-02-07,0
3,29388541,2018-05-16,0
4,29388541,2018-08-01,0


In [144]:
# Get the type count per patient per day
type_count = RX_temp[list(RX_temp.loc[:,'PATIENT_ID':'SERVICE_DATE'])+list(RX_temp.loc[:,"drug_type_AFI":'drug_type_VER'])]
type_count = pd.DataFrame(type_count.groupby(['PATIENT_ID','SERVICE_DATE']).sum())
type_count.reset_index(inplace=True)

In [109]:
# Get the total qty for patient (per drug per day)
DRUG_QTY=pd.DataFrame(RX_temp.groupby(['PATIENT_ID','SERVICE_DATE','NDC'])['DSPNSD_QTY'].sum())
DRUG_QTY.reset_index(inplace=True)
DRUG_QTY = DRUG_QTY.rename(columns={"DSPNSD_QTY":"DRUG_QTY"})
DRUG_QTY = DRUG_QTY.drop_duplicates()

drug_qty = (DRUG_QTY.pivot_table(index=['PATIENT_ID','SERVICE_DATE'],columns='NDC', values='DRUG_QTY')
       .reset_index()
       .rename_axis(None, axis=1))
drug_qty = drug_qty.fillna(0)

In [114]:
# Get the days supply information 
NDC_days = RX_temp.drop_duplicates(subset=['PATIENT_ID','SERVICE_DATE','NDC','DAYS_SUPPLY'])
days_supply = (NDC_days.pivot_table(index=['PATIENT_ID','SERVICE_DATE'],columns='NDC', values='DAYS_SUPPLY')
       .reset_index()
       .rename_axis(None, axis=1))
days_supply = days_supply.fillna(0)

In [155]:
# Calculate the number of new drug prescripted for patient X on day y.
RX_temp['New_RX'] = np.where(RX_temp.REFILL_CODE ==0,1,0)
New_RX_count = pd.DataFrame(RX_temp.groupby(['PATIENT_ID','SERVICE_DATE'])['New_RX'].sum())
New_RX_count.reset_index(inplace=True)

In [122]:
# Rename dataframe columns before merging
days_supply.columns = days_supply.columns.map(str)
drug_qty.columns = drug_qty.columns.map(str)
days_supply.columns = 'days_' + days_supply.columns.values
drug_qty.columns = 'qty_' + drug_qty.columns.values

days_supply = days_supply.rename(columns={"days_PATIENT_ID":"PATIENT_ID", "days_SERVICE_DATE":"SERVICE_DATE"})
drug_qty = drug_qty.rename(columns={"qty_PATIENT_ID":"PATIENT_ID","qty_SERVICE_DATE":"SERVICE_DATE"})

In [164]:
RX_final = type_count.merge(New_RX_count,how="left",on=['PATIENT_ID','SERVICE_DATE']).merge(days_supply,how="left",on=['PATIENT_ID','SERVICE_DATE']).merge(drug_qty,how="left",on=['PATIENT_ID','SERVICE_DATE'])

In [169]:
data_final = PX_final.merge(RX_final,how="left",on=['PATIENT_ID','SERVICE_DATE'])

In [172]:
data_final.to_csv('final_data_2.csv.gz',sep='|',header=True,index=False)

In [173]:
data_final.columns

Index(['PATIENT_ID', 'SERVICE_DATE', 'class', 'Earliest_DIAG_Date',
       'Days_Diff', 'Week_Diff', 'Year_Diff', 'CHEMOTHERAPY',
       'CYCLOPHOSPHAMIDE', 'LYMPH',
       ...
       'qty_68382038306', 'qty_69097031602', 'qty_69097094808',
       'qty_69097094903', 'qty_70121146302', 'qty_72205000660',
       'qty_72205000792', 'qty_72485020460', 'qty_72485020512',
       'qty_89141012301'],
      dtype='object', length=279)